In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
import pickle

In [2]:
all_dfs = pickle.load(open('all_dfs.pkl', 'rb'))

In [3]:
all_precincts = ['1', '5', '6', '7', '9', '10', '13', '14', '17', '18', '19', '20', '22',
       '23', '24', '25', '26', '28', '30', '32', '33', '34', '40', '41', '42',
       '43', '44', '45', '46', '47', '48', '49', '50', '52', '60', '61', '62',
       '63', '66', '67', '68', '69', '70', '71', '72', '73', '75', '76', '77',
       '78', '79', '81', '83', '84', '88', '90', '94', '100', '101', '102',
       '103', '104', '105', '106', '107', '108', '109', '110', '111', '112',
       '113', '114', '115', '120', '121', '122', '123']

In [4]:
bike_count = np.zeros(shape=len(all_precincts), dtype='int64')

for key in list(all_dfs.keys()):
    all_dfs[key] = all_dfs[key].loc[np.all(all_dfs[key].notna(), axis=1)]
    u, c = np.unique(all_dfs[key], return_counts=True)
    c = c[np.argsort(list(map(int, list(u))))]

    diff = list(map(int, list(set(all_precincts) - set(u))))
    diff.sort()
    diff = list(map(str, diff))

    for precinct in diff:
        idx = all_precincts.index(precinct)
        c = np.insert(c, idx, 0)
    
    bike_count = np.vstack([bike_count, c/2])

bike_count = bike_count[1:]

In [5]:
df = pd.DataFrame(bike_count, columns=all_precincts)

In [6]:
df.head()

,1,5,6,7,9,10,13,14,17,18,...,110,111,112,113,114,115,120,121,122,123
0,34782.5,17553.5,29745.0,10819.5,29580.5,28003.0,36118.5,44401.0,19588.0,25516.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,25988.5,14142.5,24019.0,8092.0,23946.5,21520.5,27951.5,30917.0,13964.0,16159.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,52251.0,26549.5,44865.0,15217.5,42675.0,41449.0,53198.0,62522.0,27272.5,35970.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,85821.0,40936.0,73288.5,20873.5,63335.0,64474.0,74846.5,90386.0,41948.0,55582.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120570.0,55464.5,92629.0,28127.5,79449.5,82365.0,97697.5,109153.0,51942.5,67909.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
keys = ['201' + str(Y) + '-' + str(M).rjust(2, '0') for Y in range(4, 10) for M in range(1, 13)]
df.index = list(all_dfs.keys())
diff = list(set(keys) - set(all_dfs.keys()))
diff.sort()

In [8]:
for key in diff:
    df.loc[key] = np.zeros(shape=len(all_precincts), dtype='int64')

In [9]:
df = df.sort_index()

In [10]:
df

,1,5,6,7,9,10,13,14,17,18,...,110,111,112,113,114,115,120,121,122,123
2014-01,34782.5,17553.5,29745.0,10819.5,29580.5,28003.0,36118.5,44401.0,19588.0,25516.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-02,25988.5,14142.5,24019.0,8092.0,23946.5,21520.5,27951.5,30917.0,13964.0,16159.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-03,52251.0,26549.5,44865.0,15217.5,42675.0,41449.0,53198.0,62522.0,27272.5,35970.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-04,85821.0,40936.0,73288.5,20873.5,63335.0,64474.0,74846.5,90386.0,41948.0,55582.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-05,120570.0,55464.5,92629.0,28127.5,79449.5,82365.0,97697.5,109153.0,51942.5,67909.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08,238078.5,105508.5,144391.5,59040.0,162847.5,177200.0,175849.5,179774.5,93129.0,139264.0,...,0.0,0.0,0.0,0.0,47221.0,0.0,0.0,0.0,0.0,0.0
2019-09,242694.5,106040.0,158896.0,59289.0,176154.5,188311.0,189187.5,188523.5,99361.0,142335.5,...,0.0,0.0,0.0,0.0,43972.0,0.0,0.0,0.0,0.0,0.0
2019-10,188647.0,87320.5,138270.5,49729.0,151819.0,162306.0,169653.0,180882.5,90375.0,134905.0,...,0.0,0.0,0.0,0.0,37767.0,0.0,0.0,0.0,0.0,0.0
2019-11,118964.5,62445.5,95864.5,38205.5,112308.5,111400.0,127506.5,133758.0,67734.0,96143.0,...,0.0,0.0,0.0,0.0,24315.0,0.0,0.0,0.0,0.0,0.0
